In [1]:
# Hadley Wickham 2014 yılında derli toplu veriler hakkında bir makale yazmıştır
# Bu makale ile verilerimizin şeklinin tanımlanması formalize edilmiştir
# Bu şekilde veri kümesindeki veri değerlerinin organize edilmesi için standart bir yöntem sağlanmıştır

In [14]:
import pandas as pd

data = {'name':['Daniel', 'John', 'Jane'],
        'treatment a':[None, 12, 24],
        'treatment b':[42, 31, 27]}

df = pd.DataFrame(data)
print(df)

     name  treatment a  treatment b
0  Daniel          NaN           42
1    John         12.0           31
2    Jane         24.0           27


In [6]:
# Derli toplu veri üç prensibe dayanır:
# 1) Sütunlar ayrı değişkenleri temsil eder
# 2) Satırlar tekil gözlemleri temsil eder
# 3) Her bir gözlemsel birim bir tablo oluşturur

In [16]:
# Dolayısıyla üstteki tablo bu prensiplere uymamaktadır. Çünkü sütunlar ayrı değişkenleri değil aynı değişkenin
# farklı değerlerini temsil etmektedir. Dolayısıyla aslında verilerin şekli aşağıdaki gibi olmalıdır
# Verilerimizi derli toplu hale getirmek için melt fonksiyonunu kullanıyoruz, bu işleme ise melting deniyor
# id_vars ile sabit kalmasıın istediğimiz, value_vars ile birleştirmek istediğimiz sütunları seçiyoruz

In [15]:
df = pd.melt(frame=df, id_vars='name', value_vars=['treatment a', 'treatment b'],
             var_name='treatment', value_name='result')
print(df)

     name    treatment  result
0  Daniel  treatment a     NaN
1    John  treatment a    12.0
2    Jane  treatment a    24.0
3  Daniel  treatment b    42.0
4    John  treatment b    31.0
5    Jane  treatment b    27.0


In [17]:
# İlk bakışta bu format kullanışsız gelebilir. Fakat raporlama için ve analiz için daha iyi olan formatlar vardır
# Verileri temizleme aşamasında derli toplu hale getirerek yaygın karşılaşılan veri problemlerini elimine edebiliriz
# ve gerektiğinde hızlıca verileri farklı formatlara dönüştürebiliriz

In [19]:
# Farklı süreçlerde farklı veri şekillerine ihtiyaç duyduğumuzdan melting işlemini tersine çevirebiliriz
# Melting işleminde sütunları tek bir sütuna çeviriyorduk. Tersini yapmak için ise satırlardaki her bir unique
# değeri sütunlara dönüştürürüz. Bu işlem pivoting olarak bilinir. Bu şekilde analize elverişli bir şekilden
# raporlamaya elverişli bir şekle dönebiliriz ve ya eğer satırlar bireysel gözlemleri temsil etmiyorsa özellikle
# sütunlarda birden fazla değişken tutuluyorsa pivoting yapabiliriz

In [20]:
data = {'date':['2010-01-30', '2010-01-30', '2010-02-02', '2010-02-02'],
        'element':['tmax', 'tmin', 'tmax', 'tmin'],
        'value':[27.8, 14.5, 27.3, 14.4]}

df = pd.DataFrame(data)
print(df)

         date element  value
0  2010-01-30    tmax   27.8
1  2010-01-30    tmin   14.5
2  2010-02-02    tmax   27.3
3  2010-02-02    tmin   14.4


In [21]:
# Yukarıdaki tabloda iki güne ait en yüksek ve en düşük sıcaklık değerleri bulunmaktadır
# En yüksek ve en düşük sıcaklık değerleri farklı satırlarda raporlandığı için satırlarda tekrarlanmalar oluşmuş
# Oysa biz her farklı günün farklı bir gözlem olmasını ve satırlarda her bir gözlemin tutulmasını istiyoruz
# Verileri tekrar derli toplu bir formatta tutmak için pivot metodunu kullanacağız

In [22]:
df_tidy = df.pivot(index='date', columns='element', values='value')
print(df_tidy)

element     tmax  tmin
date                  
2010-01-30  27.8  14.5
2010-02-02  27.3  14.4


In [23]:
# Index argümanı ile sabit tutacağımız sütunu, columns argümanı ile pivot yapacağımız sütunları ve values
# argümanı ile yeni oluşturulacak sütunlara değerleri sağlayacak olan sütunları belirleriz
# Eğer tekrarlamalı değerler olursa pivot fonksiyonu çalışmaz. Bu durumda pivot_table metodunu kullanmalıyız
# Birden fazla index bulunduğunda pivot işlemi sonucunda indexlerden biri gruplanabilir, bu durumda reset_index
# metodu ile grupları ayırabiliriz

In [24]:
data = {'date':['2010-01-30', '2010-01-30', '2010-02-02', '2010-02-02', '2010-02-02'],
        'element':['tmax', 'tmin', 'tmax', 'tmin', 'tmin'],
        'value':[27.8, 14.5, 27.3, 14.4, 13.8]}

df = pd.DataFrame(data)
print(df)

         date element  value
0  2010-01-30    tmax   27.8
1  2010-01-30    tmin   14.5
2  2010-02-02    tmax   27.3
3  2010-02-02    tmin   14.4
4  2010-02-02    tmin   13.8


In [26]:
import numpy as np

df_tidy = df.pivot_table(index='date', columns='element', values='value', aggfunc=np.mean)
print(df_tidy)

element     tmax  tmin
date                  
2010-01-30  27.8  14.5
2010-02-02  27.3  14.1


In [27]:
# Bir diğer yaygın veri problemi sütunların birden fazla bilgi içermesidir

In [29]:
data = {'country':['AD', 'AE', 'AF'], 'year':[2000, 2000, 2000], 'm014':[0, 2, 52], 'm1524':[0, 4, 228]}
df = pd.DataFrame(data)
print(df)

  country  year  m014  m1524
0      AD  2000     0      0
1      AE  2000     2      4
2      AF  2000    52    228


In [30]:
# Yukarıdaki tabloda cinsiyet ve yaş bilgilerinin aynı sütunlarda bulunduğu görülüyor
# m014 0-14 yaş aralığındaki erkekler anlamına gelirken m1524 15-24 yaş aralığındaki erkekler anlamına geliyor
# Dolayısıyla yaş ve cinsiyet sütunları birbirinden ayrılmalıdır. İlk olarak melt işlemi ile yaş ve cinsiyet
# sütunlarını aynı sütun içerisinde toplarız. Verileri raporlarken yine aynı format kullanılabilir fakat analiz
# esnasında değişkenlerin ayrılması daha elverişlidir. Örneğin yaş ve cinsiyeti predictor olarak kullanabiliriz
# Dolayısıyla soruları cevaplayabilmek için gerektiğinde sütunları birleştirmeli ve ya ayırmalıyız

In [32]:
df = pd.melt(frame=df, id_vars=['country', 'year'])
print(df)

  country  year variable  value
0      AD  2000     m014      0
1      AE  2000     m014      2
2      AF  2000     m014     52
3      AD  2000    m1524      0
4      AE  2000    m1524      4
5      AF  2000    m1524    228


In [33]:
# Ardından cinsiyet bilgisini cinsiyet-yaş sütunundan parse ederiz
# Bunun için sütun üzerinde str metodunu kullanarak sütuna bir string gibi davranabiliriz
# Dolayısıyla her türlü string metodunu uygulayabiliriz, split vs...

In [34]:
df['sex'] = df.variable.str[0]
print(df)

  country  year variable  value sex
0      AD  2000     m014      0   m
1      AE  2000     m014      2   m
2      AF  2000     m014     52   m
3      AD  2000    m1524      0   m
4      AE  2000    m1524      4   m
5      AF  2000    m1524    228   m
